In [1]:
!pip install  -q transformers==4.49.0 datasets==2.20.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 52.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.5.0 which is incompatible.


# **Dataset**

In [2]:
from datasets import load_dataset
dataset = load_dataset("thainq107/iwslt2015-en-vi")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/133317 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1268 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'vi'],
        num_rows: 133317
    })
    validation: Dataset({
        features: ['en', 'vi'],
        num_rows: 1268
    })
    test: Dataset({
        features: ['en', 'vi'],
        num_rows: 1268
    })
})

# **Tokenizer**

In [4]:
model_id = "facebook/mbart-large-50-many-to-many-mmt"

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

# **Preprocessing**

In [6]:
import numpy as np
MAX_LEN = 64

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["en"], max_length=MAX_LEN, truncation=True, padding="max_length"
    )

    labels = tokenizer(
        examples["vi"], max_length=MAX_LEN, truncation=True, padding="max_length"
    )

    labels_ids = labels["input_ids"]
    labels_ids = np.where(np.array(labels_ids) == tokenizer.pad_token_id, -100, np.array(labels_ids))

    return {
        "input_ids": np.array(model_inputs["input_ids"]),
        "labels": labels_ids,
    }


In [7]:
preprocessed_ds = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/133317 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [8]:
len(preprocessed_ds["train"][0]["input_ids"])

64

# **Evaluate**

In [9]:
!pip install -q evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.7 MB/s eta 0:00:00


In [10]:
import numpy as np
import evaluate
metric = evaluate.load("sacrebleu")

def preprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    torch.cuda.empty_cache()  # Giải phóng bộ nhớ trước khi tính toán

    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = preprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    del preds, labels, decoded_preds, decoded_labels  # Xóa biến không cần thiết
    torch.cuda.empty_cache()  # Dọn dẹp bộ nhớ lần nữa

    return {"bleu": result["score"]}


# **Model**

In [11]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [14]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [15]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    output_dir="mbart50/mt-en-vi",
    logging_dir="logs",
    predict_with_generate=True,
    logging_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=1,
    load_best_model_at_end=True,
    remove_unused_columns=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=preprocessed_ds["train"],
    data_collator=data_collator,
    # tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    eval_dataset=preprocessed_ds["validation"],
    processing_class=tokenizer
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [19]:
trainer.train()

Step,Training Loss,Validation Loss,Bleu
500,1.188700,1.296070,34.211824
1000,1.142200,1.280743,34.692906
1500,1.127000,1.268904,34.762503


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Step,Training Loss,Validation Loss,Bleu
500,1.188700,1.296070,34.211824
1000,1.142200,1.280743,34.692906
1500,1.127000,1.268904,34.762503
2000,1.121500,1.257672,34.572596
2500,1.031000,1.272798,35.068307
3000,1.012800,1.263245,34.809990
3500,1.018400,1.256533,34.772100
4000,1.013100,1.256362,34.958497


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=4168, training_loss=1.0790215600246202, metrics={'train_runtime': 1860.4481, 'train_samples_per_second': 143.317, 'train_steps_per_second': 2.24, 'total_flos': 3.611440490899046e+16, 'train_loss': 1.0790215600246202, 'epoch': 2.0})

In [22]:
trainer.push_to_hub()

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hai2131/mt-en-vi/commit/74997647e6ad62d2e83ad805f9c335a9e578429b', commit_message='End of training', commit_description='', oid='74997647e6ad62d2e83ad805f9c335a9e578429b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hai2131/mt-en-vi', endpoint='https://huggingface.co', repo_type='model', repo_id='hai2131/mt-en-vi'), pr_revision=None, pr_num=None)

# **Inference**

In [2]:
from transformers import pipeline

In [3]:
translator = pipeline(model="hai2131/mt-en-vi")

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
translated_text = translator("Success is not the key to happiness. Happiness is the key to success.", num_beams = 5)
translated_text

[{'generated_text': 'Thành công không phải là chìa khoá cho hạnh phúc . Hạnh phúc là chìa khoá cho thành công .'}]

In [6]:
translated_text = translator("The intricacies of human emotions are often lost in translation, making it challenging to convey true feelings across languages", num_beams = 5)
translated_text

[{'generated_text': 'Sự phức tạp của cảm xúc con người thường bị đánh mất trong việc dịch , khiến việc truyền tải cảm xúc thật sự qua ngôn ngữ trở nên khó khăn'}]